<p><a name="sections"></a></p>


# Pandora 
## Data Engineer Coding Challenge
### Lainey(Nan) Liu  

Getting the information as required is not difficult using jupyter notebook, however, if we are trying to get more comprehensive information, then we need something more user interactive.

Therefore, I try to use command line to have a simple user interactive for information retrieving.

#### Steps 



- <a href="#q1">A) Download </a>
- <a href="#q2">B) Clean the data </a>
- <a href="#q3">C) Ingest</a>
- <a href="#q4">D) Analysis </a>
- <a href="#q5">E) Favorite tools or techniques </a>
https://github.com/mediawiki-utilities/python-mwviews
https://github.com/Commonists/pageview-api

In [1]:
import pandas as pd
import datetime
import os
from inputdata import clean_before_ingest
import sqlite3

In [2]:
import sqlite3

In [6]:
data_path = './database/'
filename = 'wiki'

os.makedirs(data_path, exist_ok=True)

db = sqlite3.connect(data_path + filename + '.sqlite3')
db.execute('CREATE TABLE IF NOT EXISTS wikimedia (id integer primary key autoincrement not null, language STRING, page_name STRING, non_unique_views INTEGER, timestamp TEXT, last_update datetime default current_timestamp )')
db.close()

In [16]:
k

,language,page_name,non_unique_views
0,aa,Main_Page,1
1,aa,%D0%92%D0%B0%D1%81%D1%96%D0%BB%D1%8C_%D0%91%D1...,1
2,aa,Meta.wikimedia.org/wiki/Proposals_for_closing_...,1
3,aa,meta.wikimedia.org/wiki/Proposals_for_closing_...,1
4,ab,%D0%97%D0%B0%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%...,1
5,ab,%D0%B7%D0%B0%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%...,1
6,ab,windshield,1
7,ab,ab,9
8,ab,Help%3AContents,1
9,ab,Help%3AFAQ,1


In [22]:

[tuple(x) for x in k.values]

[('aa', 'Main_Page', 1),
 ('aa',
  '%D0%92%D0%B0%D1%81%D1%96%D0%BB%D1%8C_%D0%91%D1%8B%D0%BA%D0%B0%D1%9E',
  1),
 ('aa', 'Meta.wikimedia.org/wiki/Proposals_for_closing_projects/', 1),
 ('aa', 'meta.wikimedia.org/wiki/Proposals_for_closing_projects/', 1),
 ('ab',
  '%D0%97%D0%B0%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0',
  1),
 ('ab',
  '%D0%B7%D0%B0%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0',
  1),
 ('ab', 'windshield', 1),
 ('ab', 'ab', 9),
 ('ab', 'Help%3AContents', 1),
 ('ab', 'Help%3AFAQ', 1)]

In [9]:


con = sqlite3.connect(data_path + filename + '.sqlite3')
con.executemany(
    "insert into wikimedia(language, page_name, non_unique_views, timestamp) values (?, ?, ?, ?)",
    [tuple(x) for x in k.values])
con.commit()

In [8]:
con.close()

In [26]:
con.execute("create table person(firstname, lastname)")

### A) Clean the data

In [3]:
df = pd.read_csv(
    './waiting/pagecounts-20120101-000000.txt',
    sep=' ', encoding='latin-1', header = None)
df.columns = ['language', 'page_name', 'non_unique_views', 'bytes_transferred']

- detailed cleaning in inputdata.py
    - get rid of the .type in language, if we want to consider type in the future, then we should create a column just for type
    - to make the data more detailed, adding a column of timeframe
    - the page name we don't care normally has ":" in the page_name, however, if this rule does not work for some special cases, then we need to figure out another way to clean out the page_name we don't care. 
    - eventually we narrow down to 3 columns, which is going to be our scheme when it comes to ingestion.
    - WIP: find the duplicates in the page_name by using dedupe method, I've done a similar task where I wrote a python scrapy to get more information about the page to identify the duplication, which can be accurate, but time consuming. I would recommand using dedupe first. 

In [4]:
newDate = datetime.datetime(2012, 1, 1, 0)
k = clean_before_ingest(df, newDate).head(10)


In [13]:
df['date'] = newDate.strftime("%Y/%m/%d %H")

In [5]:
k

,language,page_name,non_unique_views,timestamp
0,aa,Main_Page,1,2012/01/01 00
1,aa,%D0%92%D0%B0%D1%81%D1%96%D0%BB%D1%8C_%D0%91%D1...,1,2012/01/01 00
2,aa,Meta.wikimedia.org/wiki/Proposals_for_closing_...,1,2012/01/01 00
3,aa,meta.wikimedia.org/wiki/Proposals_for_closing_...,1,2012/01/01 00
4,ab,%D0%97%D0%B0%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%...,1,2012/01/01 00
5,ab,%D0%B7%D0%B0%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%...,1,2012/01/01 00
6,ab,windshield,1,2012/01/01 00
7,ab,ab,9,2012/01/01 00
8,ab,Help%3AContents,1,2012/01/01 00
9,ab,Help%3AFAQ,1,2012/01/01 00


### B) Download

In [1]:
import gzip
import urllib.request

In [14]:
newDate = datetime.datetime(2011, 1, 23, 2)

In [20]:
newDate.strftime("%Y/%Y-%m/pagecounts-%Y%m%d-%H0000.gz")

'2011/2011-01/pagecounts-20110123-020000.gz'

In [23]:
baseURL = "http://dumps.wikimedia.org/other/pagecounts-raw/2012/2012-01/"
filename = "pagecounts-20120101-000000.gz"
outFilePath = filename[:-3]


response = urllib.request.urlopen(baseURL + filename)
with open(outFilePath, 'wb') as outfile:
    outfile.write(gzip.decompress(response.read()))

KeyboardInterrupt: 

In [28]:
import os
src = "page2012.txt"
dst = "./data/page20123.txt"
if os.path.isfile(src):
    os.rename(src, dst)

In [27]:
os.path.isfile(src)

False

# Basic Cleaning 
- the language and the project name is combined in column 1 and we want it to be separated


In [40]:
'aa'.split('.')

['aa']

In [41]:
df['language_'] = [i.split('.')[0] for i in df['language']]

AttributeError: 'float' object has no attribute 'split'

In [56]:
df = df.dropna(
    subset=['language']).reset_index(drop=True)

In [45]:
i = 12.3 
isinstance(i, float)

True

In [49]:
[j for j, x in enumerate([isinstance(i, float) for i in df.language.drop_duplicates()]) if x]

[729]

In [53]:
df.language.drop_duplicates().reset_index(drop=True)[729]

nan

In [57]:
df['type'] =  df['language'][0].split('.')[0]

In [62]:
import numpy as np

In [75]:
df['type'] = [i.split('.')[1] if len(i.split('.'))==2 else np.nan for i in df['language'] ]
df['language'] = [i.split('.')[0] for i in df['language']]

In [76]:
df.head()

,language,page_name,non_unique_views,bytes_transferred,type_null,type
0,aa,MediaWiki:Aboutpage,1,5354,aa.b,b
1,aa,MediaWiki:Exif-scenecapturetype-2,1,5379,aa.b,b
2,aa,Special:ListFiles/212.112.235.78,1,5579,aa.b,b
3,aa,Special:WhatLinksHere/MediaWiki:Number_of_watc...,1,5492,aa.b,b
4,aa,Special:WhatLinksHere/MediaWiki:Rollbacklink,1,5442,aa.b,b


In [72]:
df['language'][0].split('.')[1]

'b'

In [9]:
df[~df['page_name'].str.contains(
    ":",na = False)].sort_values(
    ['language','non_unique_views'],
    ascending=[True,False]).groupby('language').head(10).dropna(
    subset=['language']).reset_index(drop=True)

,language,page_name,non_unique_views,bytes_transferred
0,aa,%D0%92%D0%B0%D1%81%D1%96%D0%BB%D1%8C_%D0%91%D1...,1,688
1,aa,Meta.wikimedia.org/wiki/Proposals_for_closing_...,1,15010
2,aa,meta.wikimedia.org/wiki/Proposals_for_closing_...,1,756
3,aa.d,Main_Page,1,5564
4,ab,%D0%98%D1%85%D0%B0%D0%B4%D0%BE%D1%83_%D0%B0%D0...,5,121366
5,ab,%D0%90%D0%BC%D0%B5%D1%80%D0%B8%D0%BA%D0%B0_%D0...,2,20647
6,ab,%D0%90%D1%82%D3%99%D1%8B%D0%BB%D0%B0%D2%9B%D3%...,2,32764
7,ab,%D0%90%D2%A7%D1%81%D0%BD%D1%8B,2,53477
8,ab,%D0%91%D1%80%D0%B8%D1%82%D0%B0%D0%BD%D0%B8%D0%...,2,116220
9,ab,%D0%A7%D0%B8%D0%BB%D0%B8,2,55598


In [30]:
types = pd.DataFrame({'type': [i[0] for i in df[df['page_name'].str.contains(":",na = False)].page_name.str.split(':')]})

In [25]:
df.page_name.str.split(':')[0][0]

'MediaWiki'

In [32]:
types.drop_duplicates()

,type
0,MediaWiki
2,Special
6,Template
10,User
11,File
12,Image
16,Talk
19,%D0%90%D0%BC%D0%B5%D0%B4%D0%B8%D0%B0%D0%B2%D0%...
21,%D0%90%D1%88%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD
23,Wiktionary
